---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [17]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [134]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    current_state = ""
    state_towns = []

    with open('university_towns.txt') as file:
        for line in file:
            current = line[:-1]
            if current[-6:] == "[edit]":
                state = current[:-6]
                continue
            if "(" in line:
                town = current[:line.index('(')-1]
                state_towns.append([state, town])
            else:
                town = current
                state_towns.append([state, town])

    ut_df = pd.DataFrame(state_towns, columns = ["State", "RegionName"])
    
    return ut_df

# get_list_of_university_towns()

In [129]:
def get_gdp_data():
    
    gdp = pd.read_excel("gdplev.xls")
    gdp = gdp.iloc[219:, 4:7]
    gdp.columns = ["Quarter","GDP_Billions_Current", "GDP_Billions"]
    del gdp["GDP_Billions_Current"]
    gdp["GDP_Billions"].astype("float64")
    gdp["Net_Change"] = gdp.GDP_Billions - gdp.GDP_Billions.shift(1)
    gdp = gdp.reset_index()
    del gdp["index"]
    
    return gdp

In [130]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    data = get_gdp_data()

    for i in range(0, len(data)):
        if data.iloc[i]["Net_Change"] < 0 and data.iloc[i+1]["Net_Change"] < 0:
            start_index = i
            break
            
    return data.iloc[start_index].Quarter
    
# get_recession_start()

'2008q3'

In [131]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    data = get_gdp_data()

    start = get_recession_start()
    start_index = data[data["Quarter"] == start].index.tolist()[0]  # returns as int
    
    for i in range(start_index, len(data)):
        if data.iloc[i]["Net_Change"] > 0 and data.iloc[i+1]["Net_Change"] > 0:
            end_index = i+1
            break
            
    return data.iloc[end_index].Quarter

# get_recession_end()

'2009q4'

In [132]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    data = get_gdp_data()

    start = get_recession_start()
    start_index = data[data["Quarter"] == start].index.tolist()[0]  # returns as int
    end = get_recession_end()
    end_index = data[data["Quarter"] == end].index.tolist()[0]      # returns as int
    
    recession = data[(data.index >= start_index) & (data.index <= end_index)]
    min_gdp = recession["GDP_Billions"].min()
    bottom_index = recession[recession["GDP_Billions"] == min_gdp].index.tolist()[0]  # returns as int
      
    return data.iloc[bottom_index].Quarter

# get_recession_bottom()

'2009q2'

In [138]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    raw = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    # Only want state and region, from 2000-01 on
    city_info = raw.iloc[:, [1,2]]
    numbers = raw.iloc[:, 51::]
    
    # Group by Quarters
    numbers = numbers.groupby(pd.PeriodIndex(numbers.columns, freq="q"), axis=1).mean()
    
    # Cast PeriodIndex type to string, then convert Q to lowercase q
    numbers.columns = numbers.columns.to_series().astype("str")
    numbers.columns = map(str.lower, numbers.columns)

    # Merge dataframes
    cities = pd.merge(city_info, numbers, left_index=True, right_index=True)
    
    # Map state abbreviations to full names
    cities["State"] = cities["State"].map(states)
    
    # Set indicies
    cities = cities.set_index(["State","RegionName"])
    
    return cities

# convert_housing_data_to_quarters()

In [157]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    university_towns = get_list_of_university_towns()
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()

    house_prices = convert_housing_data_to_quarters().loc[:, start:end]
    house_prices = pd.merge(left=house_prices, right=university_towns, how="outer", left_index=True, right_on=["State", "RegionName"], indicator=True)
    house_prices["Price_Ratio"] = house_prices.apply(lambda row: (row["2008q3"]-row["2009q2"] ) / row["2009q2"], axis=1)

    # Subset dataframes
    uni = house_prices.query('_merge == "both"').dropna()
    non_uni = house_prices.query('_merge == "left_only"').dropna()
    
    # Price ratios
    uni_pr = uni["Price_Ratio"].mean()
    non_uni_pr = non_uni["Price_Ratio"].mean()

    # T-test
    ttest = ttest_ind(uni["Price_Ratio"], non_uni["Price_Ratio"], nan_policy="omit")

    # Price ratio comparison function
    def better():
        if uni_pr < non_uni_pr:
            return "university town"
        else:
            return "non-university town"

    # Return variables
    different = ttest[1] < 0.01
    p = ttest[1]
    better = better()

    return tuple([different, p, better])

# run_ttest()

(True, 0.0054964273536940869, 'university town')